In [1]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import os

In [2]:
fomc_transcripts = pd.read_csv("https://raw.githubusercontent.com/vtasca/fed-statement-scraping/master/communications.csv")

In [4]:
# load sentiment analysis pipeline
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

#function to preprocess and predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Predict using the model
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract logits (raw scores)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Map predicted class to sentiment labels
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[predicted_class]
    
    return sentiment

fomc_transcripts['sentiment'] = fomc_transcripts['Text'].apply(predict_sentiment)

c:\Users\limfy\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
fomc_transcripts.to_csv((os.getcwd()+"\\sentiment.txt"))